### Initialization chapter 7 - QA across documents

In [1]:
"""
Chapter 7 — Indexing pipeline (Q&A across documents)
- Load many file types from a folder (docx/pdf/txt)
- Split -> Embed -> Store in Chroma
- OpenRouter-compatible env (fail-fast)
"""

'\nChapter 7 — Indexing pipeline (Q&A across documents)\n- Load many file types from a folder (docx/pdf/txt)\n- Split -> Embed -> Store in Chroma\n- OpenRouter-compatible env (fail-fast)\n'

In [43]:
# =============================================================================
# IMPORTS
# =============================================================================
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import Docx2txtLoader, PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.chat_message_histories import ChatMessageHistory

In [3]:
# =============================================================================
# ENV SETUP (fail-fast)
# =============================================================================
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL")  # e.g. https://openrouter.ai/api/v1
if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY in .env")
if not OPENROUTER_BASE_URL:
    raise RuntimeError("Missing OPENROUTER_BASE_URL in .env")

In [25]:

# =============================================================================
# EMBEDDINGS
# =============================================================================
embeddings_model = OpenAIEmbeddings(
    api_key=OPENROUTER_API_KEY,
    base_url=OPENROUTER_BASE_URL,
    model="text-embedding-3-small",
)

# Initialize LLM
chatbot = ChatOpenAI(
    api_key=OPENROUTER_API_KEY,        # API key for authentication
    base_url=OPENROUTER_BASE_URL,      # OpenRouter endpoint URL
    model="openai/gpt-4o",             # GPT-4o model for high-quality responses
    temperature=0,                   # Low temperature for more deterministic, focused answers
)

### Indexing pipeline for RAG (chapter 7 - QA across documents)

In [5]:
# =============================================================================
# VECTOR STORE (Chroma)
# =============================================================================
vector_db = Chroma(
    collection_name="tourist_info",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)


In [6]:
# =============================================================================
# SPLITTER
# =============================================================================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
)

In [7]:
# =============================================================================
# LOADER FACTORY (extension -> loader)
# =============================================================================
loader_classes = {
    "docx": Docx2txtLoader,
    "pdf": PyPDFLoader,
    "txt": TextLoader,
}

def get_loader(filename: str):
    _, ext = os.path.splitext(filename)
    ext = ext.lstrip(".").lower()

    loader_class = loader_classes.get(ext)
    if loader_class:
        return loader_class(filename)

    raise ValueError(f"No loader available for file extension '{ext}'")

In [8]:
# =============================================================================
# INGESTION ORCHESTRATOR
# =============================================================================
def split_and_import(loader):
    docs = loader.load()  # -> list[Document]
    chunks = text_splitter.split_documents(docs)  # Document -> list[Document]
    vector_db.add_documents(chunks)  # embed + store
    print(f"{loader.__class__.__name__}: {len(chunks)} chunks ingested")

In [9]:
# =============================================================================
# INGEST A FOLDER
# =============================================================================
folder_path = "CilentoTouristInfo"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if os.path.isfile(file_path):
        try:
            loader = get_loader(file_path)
            print(f"Loader for {filename}: {loader.__class__.__name__}")
            split_and_import(loader)
        except ValueError as e:
            print(e)

print("✅ Indexing complete.")

Loader for Acciaroli.pdf: PyPDFLoader
PyPDFLoader: 6 chunks ingested
Loader for Cape Palinuro.txt: TextLoader
TextLoader: 4 chunks ingested
Loader for Casalvelino.txt: TextLoader
TextLoader: 4 chunks ingested
Loader for Cilentan coast.docx: Docx2txtLoader
Docx2txtLoader: 5 chunks ingested
Loader for Cilento Coast Map and Travel Guide.docx: Docx2txtLoader
Docx2txtLoader: 24 chunks ingested
Loader for Cilento DOC.pdf: PyPDFLoader
PyPDFLoader: 3 chunks ingested
Loader for Cilento Park.txt: TextLoader
TextLoader: 32 chunks ingested
Loader for Cilento.docx: Docx2txtLoader
Docx2txtLoader: 17 chunks ingested
Loader for Cilento.pdf: PyPDFLoader
PyPDFLoader: 9 chunks ingested
Loader for Marina di Camerota info.pdf: PyPDFLoader
PyPDFLoader: 16 chunks ingested
Loader for Marina di Camerota.docx: Docx2txtLoader
Docx2txtLoader: 9 chunks ingested
Loader for NationalParkOfCilentoAndValloDiDiano.txt: TextLoader
TextLoader: 9 chunks ingested
Loader for Parco Nazionale del Cilento.pdf: PyPDFLoader
PyPDF

### 🔍 เช็กเพิ่มแบบ “พิสูจน์ index” (แนะนำมาก)

In [41]:
docs = vector_db.similarity_search("Cilento Coast", k=3)
for i, d in enumerate(docs, 1):
    print(f"\n--- hit {i} ---")
    print("source:", d.metadata.get("source"))
    print(d.page_content[:300])



--- hit 1 ---
source: CilentoTouristInfo\Cilentan coast.docx
The Cilento Coast (Italian: Costiera Cilentana) is an Italian stretch of coastline in Cilento, on the southern side of the Province of Salerno. It is situated between the gulfs of Salerno and Policastro, extending from the municipalities of Capaccio-Paestum in the north-west, to Sapri in the south-e

--- hit 2 ---
source: CilentoTouristInfo\Cilento Park.txt
Cilento we mean today the southernmost part of the province of Salerno: a strip of plains south of the Sele river and the mountainous bastion between Agropoli, Sapri and Vallo di Diano. Mountains that reach almost 2000 meters and 100 km of mainly rocky coast dotted with bays and coves. The toponym h

--- hit 3 ---
source: CilentoTouristInfo\Cilento Park.txt
Cilento sea
Una caratteristica grotta lungo la costa di Marina di Camerota


In [10]:
print(vector_db._collection.count())


212


###  Generation pipeline for RAG (chapter 7 - QA across documents)

In [38]:
# =============================================================================
# GENERATION PIPELINE (Chapter 7 — Q&A / Retrieval + Generation)
# - Goal: Ask a question using stored documents in vector_db
# - Flow (Figure 7.4 / 7.5):
#   question -> retriever -> (top-k Documents)
#   Documents -> format_docs -> context string
#   {question, context} -> PromptTemplate -> prompt instance
#   prompt -> Chat model (LLM) -> answer
# =============================================================================

# =============================================================================
# IMPORTS (Prompt + Runnable)
# =============================================================================
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# =============================================================================
# 1) PROMPT TEMPLATE (Figure 7.5: Prompt parameters -> PromptTemplate)
# - {context}: retrieved text chunks (we will format them into a string)
# - {question}: user query
# - guardrails: "don't make up" + "3 sentences max" to reduce hallucination
# =============================================================================
rag_prompt_template = """Use the following pieces of context
to answer the question at the end.
If you don't know the answer, just say that you don't know,
don't try to make up an answer.
Use three sentences maximum and keep the
answer as concise as possible.

{context}

Question: {question}
Helpful Answer:"""

# Create a LangChain PromptTemplate object from the raw template string
rag_prompt = PromptTemplate.from_template(rag_prompt_template)

# =============================================================================
# 2) RETRIEVER (Figure 7.4: Q&A orchestrator -> Retriever -> VectorStore)
# - vector_db.as_retriever() wraps the vector store into a Retriever interface
# - k=4 means: fetch top 4 most similar chunks
# =============================================================================
retriever = vector_db.as_retriever(search_kwargs={"k": 4})

# =============================================================================
# 3) QUESTION FEEDER (RunnablePassthrough)
# - This passes the original question forward in the chain unchanged
# - So it can fill the {question} slot in the prompt
# =============================================================================
question_feeder = RunnablePassthrough()

# =============================================================================
# 4) CONTEXT FORMATTER (Document list -> string)
# - Retriever returns: list[Document]
# - PromptTemplate expects: {context} as a string
# - We also include "source" from metadata so the model can cite it
# =============================================================================
def format_docs(docs):
    return "\n\n".join(
        f"source: {d.metadata.get('source')}\n{d.page_content}"
        for d in docs
    )

# =============================================================================
# 5) RAG CHAIN (Figure 7.5: parameters -> prompt -> chat model)
#
# - The left dict builds the prompt parameters:
#   - "context": question -> retriever -> list[Document] -> format_docs -> string
#   - "question": question -> passthrough -> string
#
# - Then:
#   parameters -> rag_prompt (creates prompt instance) -> chatbot (generates answer)
# =============================================================================
rag_chain = (
    {"context": retriever | format_docs, "question": question_feeder}
    | rag_prompt
    | chatbot
)

# =============================================================================
# 6) EXECUTION HELPER
# - invoke(question) runs the whole chain:
#   retrieval -> prompt building -> generation
# =============================================================================
def execute_chain(chain, question):
    return chain.invoke(question)

# =============================================================================
# 7) ASK A QUESTION
# - The question includes: "Also tell me the source."
# - The model can cite sources because we injected "source: ..." into {context}
# =============================================================================
question = """Where was Poseidonia and who renamed
it to Paestum? Also tell me the source."""

answer = execute_chain(rag_chain, question)

# =============================================================================
# 8) OUTPUT
# - ChatOpenAI returns an AIMessage
# - The text you want is in answer.content
# =============================================================================
print(answer.content)


Poseidonia was located on the coasts of Cilento and was later renamed Paestum by the Lucanians. Source: CilentoTouristInfo\Cilento Park.txt.


In [39]:
question = """And then, what they do?
Tell me only if you know.
Also tell me the source"""
answer = execute_chain(rag_chain, question)

In [40]:
print(answer.content)

I don't know.


In [46]:
"""
Chapter 7.4 — RAG Chain with Chatbot Memory (compatible with your codebase)
- Assumes you already have:
  - vector_db (Chroma)
  - chatbot (ChatOpenAI using OPENROUTER_API_KEY + OPENROUTER_BASE_URL)
- Adds:
  - ChatPromptTemplate (message-based prompt)
  - ChatMessageHistory (memory)
  - RunnableLambda(get_messages) to feed latest history each turn
"""

# =============================================================================
# IMPORTS
# =============================================================================
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.chat_message_histories import ChatMessageHistory

# =============================================================================
# 1) MEMORY (chat history container)
# =============================================================================
chat_history_memory = ChatMessageHistory()

# =============================================================================
# 2) RETRIEVER (use your vector_db)
# =============================================================================
retriever = vector_db.as_retriever(search_kwargs={"k": 4})
question_feeder = RunnablePassthrough()

# =============================================================================
# 3) CONTEXT FORMATTER (list[Document] -> string with sources)
# =============================================================================
def format_docs(docs):
    return "\n\n".join(
        f"source: {d.metadata.get('source')}\n{d.page_content}"
        for d in docs
    )

# =============================================================================
# 4) CHAT-BASED PROMPT (with history placeholder)
# - "placeholder" role is used for inserting message history dynamically
# - retrieved_context is injected as an assistant message (same idea as textbook)
# =============================================================================
rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant, world-class expert in Roman and Greek history, especially in towns located in southern Italy. "
            "Answer ONLY using what is provided in the retrieved context. If you don't know, say you don't know. "
            "Use three sentences maximum and keep the answer concise. "
            "When asked for sources, cite them from lines starting with 'source: ...'.",
        ),
        ("placeholder", "{chat_history_messages}"),
        ("assistant", "{retrieved_context}"),
        ("human", "{question}"),
    ]
)

# =============================================================================
# 5) FEED LATEST HISTORY EACH INVOKE (IMPORTANT)
# - RunnableLambda ensures we read chat_history_memory.messages at runtime (not snapshot)
# =============================================================================
def get_messages(_):
    return chat_history_memory.messages

# =============================================================================
# 6) RAG CHAIN (question -> retriever -> format -> prompt -> chatbot)
# =============================================================================
rag_chain = (
    {
        "retrieved_context": retriever | format_docs,
        "question": question_feeder,
        "chat_history_messages": RunnableLambda(get_messages),
    }
    | rag_prompt
    | chatbot
)

# =============================================================================
# 7) EXECUTE WITH MEMORY (update history per turn)
# - Store user question
# - Invoke chain
# - Store AI answer (use answer.content)
# =============================================================================
def execute_chain_with_memory(chain, question: str):
    chat_history_memory.add_user_message(question)

    answer = chain.invoke(question)  # -> AIMessage (ChatOpenAI)
    chat_history_memory.add_ai_message(answer.content)

    return answer


In [49]:
# =============================================================================
# 8) TEST
# =============================================================================
q1 = "Where was Poseidonia and who renamed it to Paestum? Also tell me the source."
a1 = execute_chain_with_memory(rag_chain, q1)
print(a1.content)

Poseidonia was located on the coasts of the Cilento region in southern Italy. It was renamed to Paestum after being occupied by the Lucanians. 

source: CilentoTouristInfo\Velia.pdf


In [48]:
q2 = "And then what did they do? Also tell me the source."
a2 = execute_chain_with_memory(rag_chain, q2)
print(a2.content)

I don't know. The retrieved context does not provide further information on what happened after the Lucanians renamed Poseidonia to Paestum.
